# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
!pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv("../output_data/cities.csv")
del cities_df['Unnamed: 0'] 
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kaniama,-7.5667,24.1833,84.15,28,5,2.68,CD,1622040739
1,iranshahr,27.2025,60.6848,107.94,9,0,18.41,IR,1622040739
2,geraldton,-28.7667,114.6000,61.21,41,100,14.63,AU,1622040739
3,yellowknife,62.4560,-114.3525,42.33,45,20,5.01,CA,1622040570
4,tasiilaq,65.6145,-37.6368,36.28,97,100,40.00,GL,1622040740


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Use Lat and Lng and Humidity
humidity=cities_df["Humidity"].to_list() 
locations=cities_df[["Lat","Lng"]]

#Create a Humidity Heatmap Layer
fig = gmaps.figure(center=(20.0, 30.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2) 
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions: (80> max temp; *F> 70) and (Wind speed <10 mph) and (cloudiness = 0)
#Drop any rows with null values
#Ideal weather should include:

clouds = cities_df['Cloudiness']
temp = cities_df['Max Temp']
wind = cities_df['Wind Speed']
hotel_df = cities_df.loc[(clouds == 0) & (temp > 70) & (temp < 80) & (wind < 10)]
hotel_df.dropna()
hotel_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,limulunga,-15.0969,23.1376,79.16,23,0,5.50,ZM,1622040740
210,tortoli,39.9256,9.6572,77.99,49,0,8.05,IT,1622040797
303,ilhabela,-23.7781,-45.3581,72.41,62,0,4.43,BR,1622040823
382,sesheke,-17.4759,24.2968,77.92,23,0,4.38,ZM,1622040845
399,arcachon,44.5875,-1.0519,72.41,59,0,8.05,FR,1622040850
410,dorolt,47.8500,22.8167,73.42,28,0,3.87,RO,1622040853
423,mwinilunga,-11.7358,24.4293,72.90,38,0,4.65,ZM,1622040857
443,mongu,-15.2484,23.1274,79.41,23,0,5.53,ZM,1622040862
456,tambopata,-12.7333,-69.1833,75.09,83,0,1.21,PE,1622040866
464,jiuquan,39.7432,98.5174,70.75,15,0,5.44,CN,1622040868


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
#hotel_df = pd.DataFrame(Cities)
hotel_df["Hotel Name"]=""

#Set parameters to seasrch for hotels within 5000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_vicinity = []
print(f"Search results: \n")

for index_city in hotel_df.index:
    #print
    Coordinates = f"{hotel_df.at[index_city, 'Lat']},{hotel_df.at[index_city,'Lng']}"
    Search = "hotel%"
    Radius = 5000
    Types = "hotel%"
    Final_URL=f"{base_url}?location={Coordinates}&radius={Radius}&type={Types}&keyword={Search}&key={g_key}"
    response = requests.get(Final_URL).json()
    print(response)
    #print Final URL
    if response['status'] == "OK":
        print(f"→ City: {hotel_df.at[index_city,'City']}, Nearest Hotel (5000 m): {response['results'][0]['name']}, Vicinity: {response['results'][0]['vicinity']}")
        hotel_df.at[index_city, 'Hotel Name'] = response['results'][0]['name']
    else:
        print(f"-> City: {hotel_df.at[index_city,'City']}: {response['status']}")
    

c:\users\guest2\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Search results: 

{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started', 'html_attributions': [], 'results': [], 'status': 'REQUEST_DENIED'}
-> City: limulunga: REQUEST_DENIED
{'html_attributions': [], 'next_page_token': 'ATtYBwLuFbuFddSsnaFgRaY4QVX5LnZW1odrm4ap2rHx2mu8jmmiO5lR4N_v4IyLIykiMkLtXc6c8515rS0Qxte94ZDSwD83o9VjcOPXF0LbT1bi3NhYwqpd7YZ3MKML5PYf9bsG0_tN_i1OK9LHwQZHAplpqumhu31ibL3qzIem4HDCR7gowAJrcVt-mg1oShVCokgkUhiNGwxjBAldRTHEufr6f1n89voORaAHFzwD3MPlywSPwaNh7jLH8Als2hzozSnDKUrBLvU-gJGfTV_QnaAV9nqVpqIitcso3crt1EtGYx5loLuSHk1xooZxSwHNKWnrjc6IFwtw19EftNzl9LCogdCtr2y15QUPR3FjDRkDf1jMMnV8g8L6RBXj2_ukyTWRwSdIO6jBP4e8zCCpBGktnGro3HkOv8sdxC1iHLJjkY6LWngVvn5MpuIE15_okw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 39.9284924, 'lng': 9.7092128}, 'viewport': {'northeast': {'lat': 39.92984752989272, 'lng': 9.7105

{'html_attributions': [], 'next_page_token': 'ATtYBwLgJbS7l3A0gJ5tUX4o-TE9UGDOszcBMzXULflZoNbUtpc5ggyuDFlFZNVlcYMM8nWFiOc8JmXrZLWk7vFsoNt4F3voKY_1goUtORovwrGzw98_92MOKDuH1dWOArx3ZClJhb3qe4dDHGByCFhXXPwWPr37SfOyBEuljLeXJ6utQ9uT33whROfEE4hz5oyhACfMnlnXvwx_dhh0KXJfrqE883ej15lhHYhgRiedruYLzUA0hHhpZWCUBHIysbpxZmuduqyvG-zJDbL8pfZ3ROqTF88GpIhdzf4ZSdS4kF9teZXKoJAK-fDZovGbrzbjWGQkVm2nWkNR1CrV8-yIzd7BMtxld51IhUT3gg4dXivh4D2JMzMY4qqGlFhNUFKalp7dyoYTYIPumkwGJZG_FF-LwxDrNHF31Me3Uvt0u3B8GP31VSIe8vkNU2bErM5-TQ', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.7799432, 'lng': -45.3555353}, 'viewport': {'northeast': {'lat': -23.77852937010728, 'lng': -45.35418287010728}, 'southwest': {'lat': -23.78122902989272, 'lng': -45.35688252989272}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Maison Joly', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1335, 'html_attributions': ['<a href="https://maps.googl

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -17.486067, 'lng': 24.286402}, 'viewport': {'northeast': {'lat': -17.48491792010728, 'lng': 24.28718387989272}, 'southwest': {'lat': -17.48761757989272, 'lng': 24.28448422010727}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Protea Hotel by Marriott Zambezi River Lodge', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1366, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103123982016399729121">Protea Hotel Zambezi River Lodge</a>'], 'photo_reference': 'ATtYBwJe9bnIHAGTQjG9FTahImR44NOa0o56Y--w2jgu40GavA_5pfotZ-78viUiA3T_htRjNEQMWdAg5Cu1deW71Otuc0ljZP5eqk3XYXGBLE5PRKOtLXuTtQYKa5hlLtu0yj2C4IcvKzT8Das2CfguOjwHoLlaoB0nWbr1mAed2r7CaRsk', 'width': 2048}], 'place_id': 'ChIJ12KShl8kWhkR1DDLIP5vUMc', 'plus_code': {'compound_code': 'G77P+HH Katima Mulilo, Namibia', 'global_code': '5GJ6G77P+HH'}, 'rating': 3.7,

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 44.6164606, 'lng': -1.0973264}, 'viewport': {'northeast': {'lat': 44.61794382989272, 'lng': -1.095944470107278}, 'southwest': {'lat': 44.61524417010727, 'lng': -1.098644129892722}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'ibis Styles Arcachon Gujan Mestras (Opening June 2019)', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113479737469862042246">A Google User</a>'], 'photo_reference': 'ATtYBwKWab0tIH6OEDQOVYAYzSm_ZIU3Ssj9zKcJfZFlFtwuR1bHSdFW1bxzauUA80PzLHtXanb4aqRpe8AqtV0zNAKlWn18zUdjpthXjPkX0ENssluDLWVXhP5TX6D0niET12095uDllNmfXkRZ1Xp7SS3Mnb_y2GGMFG9sYx4QIjTVQXsj', 'width': 5184}], 'place_id': 'ChIJ3e0SyNSbVA0RQiiIhA77kTw', 'plus_code': {'compound_code': 'JW83+H3 Gujan-Mestras, France', 'global_code': '8CPWJW83+H3'}, 'rating': 4.5, 'referen

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 47.8361659, 'lng': 22.8190615}, 'viewport': {'northeast': {'lat': 47.83770367989272, 'lng': 22.82072412989272}, 'southwest': {'lat': 47.83500402010728, 'lng': 22.81802447010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Coral', 'opening_hours': {'open_now': True}, 'photos': [{'height': 640, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112761131795153725106">A Google User</a>'], 'photo_reference': 'ATtYBwLrSaE2dhjxWvfTE5BQdVVG-PgbQgRlgsu_QuPy2mbG46JmKNz3aejszUgdLIDx4IJ2n785yJqRnxlnMIFJf4p2rpvnHB9xPMASmpKxPONa4_Z6Hg7AlAG2TSxaxvcMrMU_tM2HOyXvEBMb0cUIOlLTHdYhrEP_O5y6lI5Mrzxm7Pu4', 'width': 960}], 'place_id': 'ChIJG9S88Z0POEcRqcAkOYmSQk4', 'plus_code': {'compound_code': 'RRP9+FJ Satu Mare, Romania', 'global_code': '8GV4RRP9+FJ'}, 'rating': 3.4, 'reference': 'ChIJG9S88Z0POEcRqcAkOYmSQk4', 'scope': 'GOOG

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
-> City: tambopata: ZERO_RESULTS
{'html_attributions': [], 'next_page_token': 'ATtYBwLFQeOgZMsx-v2l8QWtW8MhRb1OGQIy1PzHnHf-RbGliqYpOGeBzHymXcNIV__aNw84UhD1ZLaXdFZKqzkqBBWXouIbaK3pap32olYJQLW7Gz0H-t6OVMD9pmZ1RnJSY2z68Fj1_uvACyV9k4YTSyKonEUa_62EcZMD6zbpHxlZwJdeM8yDMHPfd2C0vEr-eMlWpuVadFn7ja9aW_O8W3j-PZAOjuU8-VBmGBwvDDEsYnvXiYh_TLLZYGC-MqTPKhZJjjiwl7u498TNnevTeJYlYIzhRP0Ku7vQFZ796R15eku2y404WNuH309Zv1GajMiItUWICDm6q2jXB0uKp1iYB2FG06m6tlpovduAGtTNrl0VUBAoXSWxTsUyCSSO9Xugc8lGt-3zxpb_hTcSpORdb4XRVfEVL4ND7T94JOsJyqV3__OUU4Jd1EBffcUWXw', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 39.737397, 'lng': 98.507802}, 'viewport': {'northeast': {'lat': 39.73872152989272, 'lng': 98.50919167989272}, 'southwest': {'lat': 39.73602187010728, 'lng': 98.50649202010727}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Jiuquan Hotel', 'photos': [{'height': 3024,

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -13.5459509, 'lng': 23.1003595}, 'viewport': {'northeast': {'lat': -13.54459137010728, 'lng': 23.10181787989272}, 'southwest': {'lat': -13.54729102989272, 'lng': 23.09911822010728}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Royal Kutachika Lodge', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113510469640868869434">stefan mynhardt</a>'], 'photo_reference': 'ATtYBwIUXjCQ-NrxtOY6ZCBH52njpdaBBKHo_9Wy4-kSWL15FybFgWY65cfv4YHQS1DHkjvANrmrUP73qEhCBpEUDER9AO2GwFdfZ46NPyKTu6bHCsOJIQN4QwE9vqkeTnLCFZM8w1QZXS70wRna_vS54_U5XL5P4kHQklEypteMwfdC21zF', 'width': 2160}], 'place_id': 'ChIJJdI6aLQ4YhkRwqqXp46FSvc', 'plus_code': {'compound_code': 'F432+J4 Zambezi, Zambia', 'global_code': '5GR5F432+J4'}, 'rating': 3.9, 'reference': 'ChIJJdI6aLQ4YhkRwqqXp46FSvc', 

In [33]:
#Hotel search result
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
210,tortoli,39.9256,9.6572,77.99,49,0,8.05,IT,1622040797,Arbatax Park Resort
303,ilhabela,-23.7781,-45.3581,72.41,62,0,4.43,BR,1622040823,Hotel Maison Joly
382,sesheke,-17.4759,24.2968,77.92,23,0,4.38,ZM,1622040845,Protea Hotel by Marriott Zambezi River Lodge
399,arcachon,44.5875,-1.0519,72.41,59,0,8.05,FR,1622040850,ibis Styles Arcachon Gujan Mestras (Opening Ju...
410,dorolt,47.8500,22.8167,73.42,28,0,3.87,RO,1622040853,Hotel Coral


In [31]:
hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == ''].index, inplace=True)
hotel_df

c:\users\guest2\anaconda3\envs\pythondata\lib\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
210,tortoli,39.9256,9.6572,77.99,49,0,8.05,IT,1622040797,Arbatax Park Resort
303,ilhabela,-23.7781,-45.3581,72.41,62,0,4.43,BR,1622040823,Hotel Maison Joly
382,sesheke,-17.4759,24.2968,77.92,23,0,4.38,ZM,1622040845,Protea Hotel by Marriott Zambezi River Lodge
399,arcachon,44.5875,-1.0519,72.41,59,0,8.05,FR,1622040850,ibis Styles Arcachon Gujan Mestras (Opening Ju...
410,dorolt,47.8500,22.8167,73.42,28,0,3.87,RO,1622040853,Hotel Coral
423,mwinilunga,-11.7358,24.4293,72.90,38,0,4.65,ZM,1622040857,Kajinga lodge
443,mongu,-15.2484,23.1274,79.41,23,0,5.53,ZM,1622040862,Acamms Gardens Lodge Mongu
464,jiuquan,39.7432,98.5174,70.75,15,0,5.44,CN,1622040868,Jiuquan Hotel
515,zambezi,-13.5432,23.1047,78.69,26,0,5.23,ZM,1622040882,Royal Kutachika Lodge
579,grinnell,41.7430,-92.7224,72.79,59,0,1.01,US,1622040675,"Country Inn & Suites by Radisson, Grinnell, IA"


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template - add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
#Need the locations and hotel_info from cell just before this
markers = gmaps.marker_layer(locations, info_box_content=hotel_info) 

#Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))